In [10]:
import collections
import pandas as pd
import peppi_py
import melee

from slippi_db import upload_lib, test_peppi, preprocessing
from slippi_db.test_peppi_ray import TEST_KEY

In [2]:
ENV = 'prod'

In [11]:
prod_test_db = upload_lib.get_db(ENV, TEST_KEY)

In [12]:
infos = list(prod_test_db.find({}))

In [13]:
df = pd.DataFrame(infos)

In [14]:
by_message = collections.defaultdict(list)
for info in infos:
    if not info['passed']:
        by_message[info['message']].append(info)

In [16]:
%mkdir -p peppi_test

In [17]:
local_files = {}
for message in by_message:
    info = by_message[message][0]
    path = upload_lib.download_slp_locally(ENV, info['key'], dir='peppi_test')
    local_files[message] = path

Downloaded slp to peppi_test/Game_20190309T113404.slp
Downloaded slp to peppi_test/21_35_46 [NOAH] Ice Climbers + [TITP] Captain Falcon (FD).slp
Downloaded slp to peppi_test/Game_20210717T040112.slp
Downloaded slp to peppi_test/Slippi_Game_20210121T172507.slp


In [18]:
print(local_files.keys())

dict_keys(["<class 'KeyError'>: 'jumps'", "AssertionError: ('p0', 'action')", "<class 'ValueError'>: 61 is not a valid EventType", "<class 'ValueError'>: 27 is not a valid Character"])


In [30]:
jumps_msg = "<class 'KeyError'>: 'jumps'"
event_61 = "<class 'ValueError'>: 61 is not a valid EventType"
char_27 = "<class 'ValueError'>: 27 is not a valid Character"

In [20]:
%pdb

Automatic pdb calling has been turned ON


In [31]:
local_files[char_27]

'peppi_test/Slippi_Game_20210121T172507.slp'

In [32]:
char_27_game = peppi_py.game(local_files[char_27])

In [33]:
char_27_game['metadata']

{'startAt': '2021-01-22T01:25:07Z',
 'lastFrame': 9351,
 'players': {'0': {'names': {'netplay': 'mang0', 'code': 'MANG#0'},
   'characters': {'27': 9477}},
  '1': {'names': {'netplay': 'null', 'code': 'NULL#478'},
   'characters': {'27': 9477}}},
 'playedOn': 'dolphin'}

In [27]:
preprocessing.assert_same_parse(local_files[value_61])

ValueError: 61 is not a valid EventType

> /usr/lib/python3.9/enum.py(678)__new__()
    676                 ve_exc = ValueError("%r is not a valid %s" % (value, cls.__qualname__))
    677                 if result is None and exc is None:
--> 678                     raise ve_exc
    679                 elif exc is None:
    680                     exc = TypeError(

ipdb> up
> /usr/lib/python3.9/enum.py(360)__call__()
    358         """
    359         if names is None:  # simple value lookup
--> 360             return cls.__new__(cls, value)
    361         # otherwise, functional API: we're creating a new Enum type
    362         return cls._create_(

ipdb> 
> /home/vlad/Repos/libmelee/melee/slpfilestreamer.py(57)dispatch()
     55             return None
     56 
---> 57         if EventType(self._contents[self._index]) == EventType.PAYLOADS:
     58             cursor = 0x2
     59             payload_size = self._contents[self._index+1]

ipdb> exit


In [28]:
value_61_game = peppi_py.game(local_files[value_61])

In [35]:
value_61_game['start']

{'slippi': {'version': [2, 0, 1]},
 'bitfield': [50, 1, 142, 76],
 'is_raining_bombs': False,
 'is_teams': False,
 'item_spawn_frequency': -1,
 'self_destruct_score': -1,
 'stage': 2,
 'timer': 480,
 'item_spawn_bitfield': [255, 255, 255, 255, 255],
 'damage_ratio': 1.0,
 'players': [{'port': 'P1',
   'character': 13,
   'type': 0,
   'stocks': 4,
   'costume': 2,
   'team': None,
   'handicap': 9,
   'bitfield': 192,
   'cpu_level': None,
   'offense_ratio': 0.0,
   'defense_ratio': 1.0,
   'model_scale': 1.0,
   'ucf': {'dash_back': None, 'shield_drop': None},
   'name_tag': ''},
  {'port': 'P4',
   'character': 2,
   'type': 0,
   'stocks': 4,
   'costume': 2,
   'team': None,
   'handicap': 9,
   'bitfield': 192,
   'cpu_level': None,
   'offense_ratio': 0.0,
   'defense_ratio': 1.0,
   'model_scale': 1.0,
   'ucf': {'dash_back': None, 'shield_drop': None},
   'name_tag': 'ＡＫＬＯ'}],
 'random_seed': 4013132889,
 'is_pal': False,
 'is_frozen_ps': True}

In [3]:
# p0_action_path = local_files["AssertionError: ('p0', 'action')"]
p0_action_path = 'peppi_test/21_35_46 [NOAH] Ice Climbers + [TITP] Captain Falcon (FD).slp'

In [4]:
game_rollbacks = peppi_py.game(p0_action_path, rollbacks=True)

In [9]:
game_rollbacks['frames'].field('index')[:10]

[
  -123,
  -123,
  -122,
  -122,
  -121,
  -121,
  -120,
  -120,
  -119,
  -119
]

In [5]:
game = peppi_py.game(p0_action_path)

In [6]:
# frames = game['frames']
frames = game_rollbacks['frames']
index = frames.field('index').to_numpy()
first_indices = []
next_idx = -123
for i, idx in enumerate(index):
    if idx == next_idx:
        first_indices.append(i)
        next_idx += 1

In [7]:
print(len(frames), len(first_indices))

12232 6211


In [ ]:
def iter_libmelee(path: str):
    console = melee.Console(
        is_dolphin=False,
        allow_old_version=True,
        path=path)
    console.connect()
    
    while True:
        gamestate = console.step()
        if gamestate is None:
            break
        yield gamestate

In [ ]:
states = list(iter_libmelee(p0_action_path))

In [ ]:
len(states)

In [ ]:
first_frame_indices = [index[i] for i in first_indices]

In [ ]:
all_frame_indices = set(index)

In [ ]:
len(all_frame_indices)

In [ ]:
index

In [45]:
raw_db = upload_lib.get_db(ENV, 'raw')
slp_db = upload_lib.get_db(ENV, 'slp')

def raw_from_slp_key(slp_key: str):
    slp_info = slp_db.find_one({'key': slp_key})
    raw_info = raw_db.find_one({'key': slp_info['raw_key']})
    return raw_info

In [41]:
slp_key = by_message[event_61][0]['key']

In [46]:
raw_info = raw_from_slp_key(slp_key)

In [47]:
raw_info

{'_id': ObjectId('61fe8d4f4f0535a414052c01'),
 'filename': 'Summit-11.zip',
 'key': '7b3327f86301f3205bad94548f48e843',
 'hash_method': 'md5',
 'type': 'zip',
 'stored_size': 208987744,
 'processed': True,
 'description': '/SlippiDump/Summit-11.zip',
 'dropbox_id': 'id:IDVgPhUdptgAAAAAAABG_Q',
 'dropbox_hash': '72d44646db6d9754776264405857f429ef372b7516bb3c963dbc8f683c330ac4'}

In [52]:
from slippi_db import parse_libmelee
from importlib import reload
reload(melee.console)

<module 'melee.console' from '/home/vlad/Repos/libmelee/melee/console.py'>

In [53]:
parse_libmelee.get_slp('peppi_test/21_35_46 [NOAH] Ice Climbers + [TITP] Captain Falcon (FD).slp')

-- is_valid: all not null
-- child 0 type: struct<percent: uint16, facing: bool, x: float, y: float, action: uint16, character: uint8, jumps_left: uint8, shield_strength: float, on_ground: bool, controller: struct<main_stick: struct<x: float, y: float>, c_stick: struct<x: float, y: float>, shoulder: float, buttons: struct<A: bool, B: bool, X: bool, Y: bool, Z: bool, R: bool, L: bool, START: bool, D_LEFT: bool, D_RIGHT: bool, D_DOWN: bool, D_UP: bool>>>
  -- is_valid: all not null
  -- child 0 type: uint16
    [
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      0,
      ...
      109,
      109,
      109,
      109,
      109,
      109,
      109,
      109,
      109,
      109
    ]
  -- child 1 type: bool
    [
      true,
      true,
      true,
      true,
      true,
      true,
      true,
      true,
      true,
      true,
      ...
      true,
      true,
      true,
      true,
      true,
      true,
      true,
      true,
      true,
